In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from catboost import CatBoostRegressor, Pool

In [2]:
data = pd.read_csv('data/data_embeded.csv', index_col=0, parse_dates=['created'])
comments = pd.read_csv('data/comments_embeded.csv')
submission = pd.read_csv('data/sample_solution.csv')

data = pd.merge(data, comments, left_on='id', right_on='issue_id', how='left').fillna(0)

# Add features

In [3]:
data['wd'] = data['created'].dt.weekday
data['hour'] = data['created'].dt.hour

data['key_code'] = data['key'].apply(lambda x: str(x).split('-')[0])
data['communication'] = data['summary'].apply(lambda x: 'communication' in x).astype(int)

## PCA

In [4]:
# Task PCA

pca = PCA(n_components=30, random_state=42)
features = [str(x) for x in range(768)]
data_pca = pca.fit_transform(data[features])
data_pca = pd.DataFrame(data_pca)
data_pca.index = data.index
data_pca = data_pca.add_prefix('pca_')

data = pd.concat([data, data_pca], axis=1)

In [5]:
# Comments PCA

pca = PCA(n_components=20, random_state=42)
features = ['comments_' + str(x) for x in range(768)]
data_pca = pca.fit_transform(data[features])
data_pca = pd.DataFrame(data_pca)
data_pca.index = data.index
data_pca = data_pca.add_prefix('pca_comments_')

data = pd.concat([data, data_pca], axis=1)

# Split data

In [6]:
target = 'overall_worklogs'

test_data = data[data[target] == -1]
train_data = data[data[target] > -1].sort_values(['communication', target, 'created', 'project_id', 'key_code'])
train_data.index = range(train_data.shape[0])
train_data

,id,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs,position_x,position_y,...,pca_comments_10,pca_comments_11,pca_comments_12,pca_comments_13,pca_comments_14,pca_comments_15,pca_comments_16,pca_comments_17,pca_comments_18,pca_comments_19
0,730910,2018-05-22 06:22:47.443,BALT-40,swap places select a template with a topic,13,197,197,60,web разработчик,web разработчик,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
1,730192,2018-11-14 10:32:18.948,BALT-762,added terraform config to create kms key and a...,13,210,210,60,директор по it,директор по it,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
2,726278,2020-03-24 08:17:18.389,BALT-4676,check context menu,13,208,33,60,web разработчик,web разработчик,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
3,730915,2018-05-22 06:16:44.070,BALT-35,"rename ""your topic"" in ""choose your class""",13,197,197,120,web разработчик,web разработчик,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
4,730912,2018-05-22 06:19:32.828,BALT-38,remove qualification from main list,13,197,197,120,web разработчик,web разработчик,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9584,823985,2019-08-14 15:29:30.000,FPY-124,communication soldering iron,29,384,384,729000,web разработчик,web разработчик,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
9585,729633,2019-06-25 12:32:21.254,BALT-1321,communications and planning,13,1,10,2097720,missing,web разработчик,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
9586,728296,2019-11-06 10:09:07.899,BALT-2658,communications and planning,13,1,10,2328360,missing,web разработчик,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
9587,727996,2019-11-30 05:22:23.728,BALT-2958,communications,13,193,193,3393960,менеджер проектов,менеджер проектов,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979


# Delete outliers

In [7]:
# Display outliers

remove_ids = [727996, 724450, 727714, 700178, 689558, 686977, 823985, 730147, 
              923360, 689590, 722732, 689154, 824367, 682136]
remove_ids += train_data.loc[(train_data['assignee_id'] == 1) & (train_data['creator_id'] == 10), 'id'].to_list()
remove_ids += train_data.loc[(train_data['assignee_id'] == 58) & (train_data['creator_id'] == 58), 'id'].to_list()

train_data[train_data['id'].isin(remove_ids)].sort_values('overall_worklogs')

,id,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs,position_x,position_y,...,pca_comments_10,pca_comments_11,pca_comments_12,pca_comments_13,pca_comments_14,pca_comments_15,pca_comments_16,pca_comments_17,pca_comments_18,pca_comments_19
16,729718,2019-06-11 07:38:48.729,BALT-1236,give access to ssh of staging,13,1,10,300,missing,web разработчик,...,0.391457,0.850506,0.224292,-0.696926,-1.570101,0.325283,0.251527,-0.529438,-0.982430,-1.003414
22,729404,2019-07-17 06:20:57.476,BALT-1550,give access to teacherly repository on the gitlab,13,1,10,300,missing,web разработчик,...,0.302703,0.380628,0.618980,-1.287057,-1.311682,-1.111611,-1.122638,1.836483,-0.035554,-0.810953
30,728883,2019-09-09 08:06:08.531,BALT-2071,branch need to be protected,13,1,10,300,missing,web разработчик,...,-0.000085,0.012021,-0.001180,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
32,728458,2019-10-21 06:23:16.586,BALT-2496,need to be able to raise staging from themes &...,13,1,10,300,missing,web разработчик,...,-0.335568,1.564822,0.111538,-0.493907,0.153866,-0.234000,0.259314,0.003346,0.339946,-1.583033
159,729054,2019-08-23 06:24:00.121,BALT-1900,provide staging database-dump,13,1,10,600,missing,web разработчик,...,-0.000085,0.012021,-0.001180,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9537,687555,2020-04-02 12:45:35.000,XXO-315,all calls,33,58,58,1707600,менеджер проектов,менеджер проектов,...,-0.000085,0.012021,-0.001180,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
9585,729633,2019-06-25 12:32:21.254,BALT-1321,communications and planning,13,1,10,2097720,missing,web разработчик,...,-0.000085,0.012021,-0.001180,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
9586,728296,2019-11-06 10:09:07.899,BALT-2658,communications and planning,13,1,10,2328360,missing,web разработчик,...,-0.000085,0.012021,-0.001180,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
9587,727996,2019-11-30 05:22:23.728,BALT-2958,communications,13,193,193,3393960,менеджер проектов,менеджер проектов,...,-0.000085,0.012021,-0.001180,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979


In [8]:
# Remove outliers from train data

mask = (train_data['id'].isin(remove_ids))

train_data = train_data[~mask]
train_data.index = range(train_data.shape[0])

# Local validation data split

In [9]:
val_step = 9
val_ids = [x for x in range(train_data.shape[0]) if x % val_step == 0]
train_ids = [x for x in range(train_data.shape[0]) if x % val_step != 0]
print(val_ids[:5])

train = train_data.loc[train_ids]
val = train_data.loc[val_ids]

display(val.head(3))
display(train.head(3))

[0, 9, 18, 27, 36]


,id,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs,position_x,position_y,...,pca_comments_10,pca_comments_11,pca_comments_12,pca_comments_13,pca_comments_14,pca_comments_15,pca_comments_16,pca_comments_17,pca_comments_18,pca_comments_19
0,730910,2018-05-22 06:22:47.443,BALT-40,swap places select a template with a topic,13,197,197,60,web разработчик,web разработчик,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
9,730851,2018-07-23 07:23:58.380,BALT-99,release elastic ip 52.70.203.51 id: eipalloc-9...,13,210,210,300,директор по it,директор по it,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
18,729472,2019-07-11 09:03:58.067,BALT-1482,task planning,13,86,86,300,web разработчик,web разработчик,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979


,id,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs,position_x,position_y,...,pca_comments_10,pca_comments_11,pca_comments_12,pca_comments_13,pca_comments_14,pca_comments_15,pca_comments_16,pca_comments_17,pca_comments_18,pca_comments_19
1,730192,2018-11-14 10:32:18.948,BALT-762,added terraform config to create kms key and a...,13,210,210,60,директор по it,директор по it,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
2,726278,2020-03-24 08:17:18.389,BALT-4676,check context menu,13,208,33,60,web разработчик,web разработчик,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979
3,730915,2018-05-22 06:16:44.070,BALT-35,"rename ""your topic"" in ""choose your class""",13,197,197,120,web разработчик,web разработчик,...,-0.000085,0.012021,-0.00118,-0.008447,0.002454,0.001499,0.000974,-0.000365,-0.001921,0.001979


# Catboost

In [10]:
class Model():
    def __init__(self, n_models, model_params=None, text_features=None):
        self.n_models = n_models
        self.models = []
        self.model_predictions = []
        self.predictions_summary = None
        self.model_params = model_params 
        self.text_features = text_features
        
    def fit(self, X, y):
        for model_number in range(self.n_models):
            print(model_number)
            
            # Model params
            objective = self.model_params['objective']
            iterations = self.model_params['iterations']
            learning_rate = self.model_params['learning_rate'] 
            random_state = self.model_params['random_state'] + self.n_models ** 2 + model_number
            early_stopping_rounds = self.model_params['early_stopping_rounds']
            cat_features = self.model_params['cat_features']
            one_hot_max_size = self.model_params['one_hot_max_size']
            
            train_idx = [x for x in range(X.shape[0]) if x%self.n_models != model_number]
            val_idx = [x for x in range(X.shape[0]) if x%self.n_models == model_number]            
            
            X_train = X.iloc[train_idx]
            y_train = y.iloc[train_idx]
            
            X_val = X.iloc[val_idx]
            y_val = y.iloc[val_idx]
          
            
            train_dataset = Pool(data=X_train, label=y_train, cat_features=cat_features, text_features=self.text_features)
            eval_dataset = Pool(data=X_val, label=y_val, cat_features=cat_features, text_features=self.text_features)  

            
            model = CatBoostRegressor(objective=objective, iterations=iterations,
                                      learning_rate=learning_rate, random_state=random_state, 
                                      one_hot_max_size=one_hot_max_size) 

            model.fit(train_dataset, eval_set=eval_dataset, silent=True, plot=True,
                      early_stopping_rounds=early_stopping_rounds)
            
            self.models.append(model.copy())   
            
        
    def predict(self, X):
        self.model_predictions = []
        for model_number in range(self.n_models):
            model = self.models[model_number]
            self.model_predictions.append(model.predict(X))  
            
        self.predictions_summary = pd.DataFrame(data=np.array(self.model_predictions)).T
        self.predictions_summary['predictions'] = (self.predictions_summary[[x for x in range(self.n_models)]]).mean(axis=1)
            
        return self.predictions_summary    

## Local Validation

In [11]:
%%time
target = 'overall_worklogs'

features = [str(x) for x in data.columns if 'pca_' in x]
features += ['vi', 'ru']
cat_features = ['project_id', 'assignee_id', 'creator_id', 'position_x', 'position_y', 'key_code', 'wd', 
                'hour', 'communication']
features += cat_features

model_params = {'iterations':2000,
                'learning_rate':0.1,
                'objective': 'RMSE',
                'random_state': 20210926, 
                'early_stopping_rounds': 100,
                'cat_features': cat_features,
                'l2_leaf_reg': 11,
                'depth': 8,
                'one_hot_max_size': 200
               }

val_predictions = []
test_predictions = []
for n_models in range(4, 10):
    ml = Model(n_models=n_models, model_params=model_params, text_features=None)
    ml.fit(train[features], train[target])
    
    val_predictions.append(ml.predict(val[features])['predictions'])
    test_predictions.append(ml.predict(test_data[features])['predictions'])

0


C:\Venvs\AltayChamp\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

1


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

...

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

8


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: total: 11min 29s
Wall time: 56 s


C:\Venvs\AltayChamp\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
C:\Venvs\AltayChamp\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [12]:
val_predictions = pd.concat(val_predictions, axis=1).mean(axis=1)
test_predictions = pd.concat(test_predictions, axis=1).mean(axis=1)

r2_score(val[target], val_predictions)

0.3128300454319428

## Submission

In [13]:
%%time
target = 'overall_worklogs'

features = [str(x) for x in data.columns if 'pca_' in x]
features += ['vi', 'ru']
cat_features = ['project_id', 'assignee_id', 'creator_id', 'position_x', 'position_y', 'key_code', 'wd', 
                'hour', 'communication']
features += cat_features

model_params = {'iterations':2000,
                'learning_rate':0.1,
                'objective': 'RMSE',
                'random_state': 20210926, 
                'early_stopping_rounds': 100,
                'cat_features': cat_features,
                'l2_leaf_reg': 11,
                'depth': 8,
                'one_hot_max_size': 200
               }

test_predictions = []
for n_models in range(4, 10):
    ml = Model(n_models=n_models, model_params=model_params, text_features=None)
    ml.fit(train_data[features], train_data[target])    
    
    test_predictions.append(ml.predict(test_data[features])['predictions'])
    
test_predictions = pd.concat(test_predictions, axis=1).mean(axis=1)

0


C:\Venvs\AltayChamp\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

1


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

...

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

8


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: total: 13min 31s
Wall time: 1min 4s


C:\Venvs\AltayChamp\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [14]:
submission['overall_worklogs'] = test_predictions
submission.to_csv('submission.csv', index=False)